In [ ]:
pip install -q pytorch-tabnet==3.1.0

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from pytorch_tabnet.tab_model import TabNetClassifier, TabNetRegressor

In [ ]:
train=pd.read_csv('../input/tabular-playground-series-feb-2021/train.csv')
test=pd.read_csv('../input/tabular-playground-series-feb-2021/test.csv')
sub=pd.read_csv('../input/tabular-playground-series-feb-2021/sample_submission.csv')

In [ ]:
cats = [c for c in train.columns if train[c].dtypes=='object']

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
for c in cats:
    le=LabelEncoder()
    le.fit(list(train[c].astype('str')) + list(test[c].astype('str')))
    train[c] = le.transform(list(train[c].astype(str))) 
    test[c] = le.transform(list(test[c].astype(str))) 
   



In [ ]:
# Load the data
X = train.drop(columns=['target']).values
# sc = StandardScaler()
# x=sc.fit_transform(x)
y = train['target'].values.reshape(-1, 1)
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.25,
    random_state=123,
)

In [ ]:
import torch

In [ ]:
tabnet_params = dict(n_d=26, n_a=26, n_steps=3, gamma=1.4,
                     lambda_sparse=0, optimizer_fn=torch.optim.Adam,
                     optimizer_params=dict(lr=2e-4, weight_decay=1e-5),
                     mask_type='entmax',
                     scheduler_params=dict(mode="min",
                                           patience=5,
                                           min_lr=1e-5,
                                           factor=0.9,),
                     scheduler_fn=torch.optim.lr_scheduler.ReduceLROnPlateau,
                     verbose=10,
                     )

In [ ]:
# train models with AutoML
re = TabNetRegressor()  #TabNetRegressor()
re.fit(
  X_train, y_train,
  eval_set=[(X_test, y_test)],
   eval_name=['train'],
    eval_metric=[ 'rmse'],
    max_epochs=250,
    patience=50,
    batch_size=1042, virtual_batch_size=128,
    num_workers=2,
    drop_last=False
)
preds = re.predict(X_test)

In [ ]:
# compute the MSE on test data
predictions = preds
print("Test MSE:", mean_squared_error(y_test, predictions,squared=False))

In [ ]:
sub['target']=re.predict(test.values)
sub.to_csv('submission.csv',index=False)